# 5.6 GPU
+ 可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算
+ 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU
+ 不经意地移动数据可能会显著降低性能
  
只要所有的数据和参数都在同一个设备上， 我们就可以有效地学习模型

In [1]:
!nvidia-smi

Tue Aug  8 09:28:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.92       Driver Version: 528.92       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P0    21W / 140W |    453MiB /  8188MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), 
# torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'))

In [3]:
# 查询GPU数量
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 5.6.2 张量与GPU
查询张量所在的设备
**注意**： 对多个项进行操作必须确保多个项都在同一个设备上，否则框架不知道
在哪存储结果或在哪执行计算

In [5]:
x = torch.tensor([1, 2, 3])
# 默认情况下，所有变量和相关的计算都分配给CPU
x.device

device(type='cpu')

### 5.6.2.1 存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
print(X)
# Pytorch当中释放X所占的显存
X = None

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [ ]:
# 如果有两个GPU 可以在第二个GPU上创建一个随机变量
Y = torch.rand(2, 3, device = try_gpu(1))
Y

### 5.6.2.2 复制
这里的意思是，要计算 `X + Y ` 当`X`位于`cuda:0`上，而`Y`位于`cuda:1`上时，需要把`X`复制到
`cuda:1`上，只有`X` 和 `Y`位于同一块GPU上，才能执行相应的计算

In [ ]:
Z = X.cuda(1) # X移动到cuda:1上
print(X) # cuda:0
print(Z) # cuda:1
Y + Z

### 5.6.2.3 旁注
在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多，这也使得并行化变得更加困难
+ 多个小操作比一个大操作糟糕得多
+ 一次执行几个操作比代码中散布的许多单个操作要好得多

## 5.6.3 神经网络与GPU
神经网络模型可以指定设备

In [8]:

net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [18]:
# X = torch.ones((2,3))
# net(X)
# 上面代码会报错 因为X默认在CPU当中，net() 在cuda:0当中 不在同一个device

X = torch.ones(2, 3, device=try_gpu())
net(X)

tensor([[0.2907],
        [0.2907]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [19]:
#  GPU对于矩阵的乘法比CPU快（小范围的要慢）
import time
import numpy as np
import torch
import os
os.environ ["KMP_DUPLICATE_LIB_OK"] ="TRUE"
# 定义矩阵的大小
matrix_size = (10000, 10000)

# 生成随机矩阵
np_matrix1 = np.random.rand(*matrix_size)
np_matrix2 = np.random.rand(*matrix_size)

# NumPy CPU矩阵乘法
start_time = time.time()
np_result = np.dot(np_matrix1, np_matrix2)
end_time = time.time()
cpu_time = end_time - start_time

# 将NumPy矩阵转换为PyTorch张量，并将其移动到GPU上（如果可用）
torch_matrix1 = torch.tensor(np_matrix1)
torch_matrix2 = torch.tensor(np_matrix2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch_matrix1 = torch_matrix1.to(device)
torch_matrix2 = torch_matrix2.to(device)

# PyTorch GPU矩阵乘法
start_time = time.time()
torch_result = torch.mm(torch_matrix1, torch_matrix2)
end_time = time.time()
gpu_time = end_time - start_time

# 输出结果
print(f"NumPy CPU 矩阵乘法耗时：{cpu_time:.6f} 秒")
print(f"PyTorch GPU 矩阵乘法耗时：{gpu_time:.6f} 秒")


NumPy CPU 矩阵乘法耗时：4.344089 秒
PyTorch GPU 矩阵乘法耗时：0.005491 秒
